### Database connection

In [9]:
from langchain_community.utilities import SQLDatabase

In [10]:
db_path = '../data/inventory.db'
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print("dialect:", db.dialect)
print("tables:", db.get_usable_table_names())

dialect: sqlite
tables: ['Orders', 'Stock']


In [11]:
records = db.run("SELECT * FROM Stock LIMIT 1")
print(records)

[('1009AA', 7210.0, 'Kg', 30, 48, 28.76326)]


In [12]:
info = db.get_table_info(["Stock"])
print(info)


CREATE TABLE "Stock" (
	id TEXT, 
	current_stock_quantity FLOAT, 
	units TEXT, 
	avg_lead_time_days BIGINT, 
	maximum_lead_time_days BIGINT, 
	unit_price FLOAT
)

/*
3 rows from Stock table:
id	current_stock_quantity	units	avg_lead_time_days	maximum_lead_time_days	unit_price
1009AA	7210.0	Kg	30	48	28.76326
1077CA	46516.0	Kg	45	70	22.9777
1083AA	48210.0	Kg	45	68	29.02
*/


In [13]:
records = db.run("SELECT * FROM Orders LIMIT 1")
print(records)

[('2022-11-19 00:00:00.000000', '3084CA', 1.0)]


In [14]:
info = db.get_table_info(["Orders"])
print(info)



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/


### LLM

In [15]:
from langchain_ollama import ChatOllama

#llm = ChatOllama(model="codegemma:7b")
llm = ChatOllama(model="qwen2.5-coder:latest")

### Text 2 SQL  Chain

Langchain provides a chain to convert the text to sql

In [16]:
from langchain.chains import create_sql_query_chain

write_query = create_sql_query_chain(llm, db)
response = write_query.invoke({"question": "How  many stocks are there?"})
print(response)

```sql
SELECT COUNT(*) FROM Stock;
```

Answer: The total number of stocks is [result from SQLQuery].


This is the prompt by default

In [17]:
write_query.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### Execute SQL query tool

In [21]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

execute_query = QuerySQLDatabaseTool(db=db)
execute_query.invoke("SELECT COUNT(*) FROM Stock")

'[(303,)]'

You can run the query predicted in the database

In [19]:
db.run("SELECT COUNT(*) FROM Stock")

'[(303,)]'

### All together

In [22]:

chain = write_query | execute_query

chain.invoke({"question": "How many stocks are there?"})

'Error: (sqlite3.OperationalError) near "```sql\nSELECT COUNT(*) as total_stocks FROM Stock;\n```": syntax error\n[SQL: ```sql\nSELECT COUNT(*) as total_stocks FROM Stock;\n```]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [24]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many stocks are there"})

'The SQL query provided has a syntax error due to the incorrect use of triple backticks (```). The correct query should not have these backticks around the SQL statement. Here is the corrected query:\n\n```sql\nSELECT COUNT(*) FROM Stock;\n```\n\nWhen executed, this query will return the total number of rows in the `Stock` table, which represents the total number of stocks.\n\nAnswer: To find out how many stocks are there, execute the following SQL query without triple backticks:\n```sql\nSELECT COUNT(*) FROM Stock;\n```\nThis query will give you the count of all entries in the `Stock` table.'